In [0]:
import torch
from torchvision import models
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
from operator import itemgetter
from heapq import nsmallest
import torchvision.transforms as transforms
from tqdm.notebook import tqdm as tqdm_notebook
from torchvision import models
import pandas as pd
import time
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [31]:
from google.colab import files
src = list(files.upload().values())[0]
open('prune.py','wb').write(src)

Saving prune.py to prune.py


7769

In [0]:
from prune import prune_layer

In [0]:
class Prunner:
    def __init__(self, model):
        self.model = model
        self.reset()
    
    def reset(self):
        self.filter_ranks = {}
        self.neuron_ranks = {}

    def forward(self, x):
        self.activations = []
        self.grad_index = 0
        self.activation_to_layer = {}

        activation_index = 0
        for layer, (name, module) in enumerate(self.model.features._modules.items()):
            x = module(x)
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                x.register_hook(self.compute_filter_rank)
                self.activations.append(x)
                self.activation_to_layer[activation_index] = layer
                activation_index += 1
        x = x.view(x.size(0), -1)
        self.n_activations = []
        self.n_grad_index = 0
        self.n_activation_to_layer = {}
        activation_index = 0
        for layer, (name, module) in enumerate(self.model.classifier._modules.items()):
            x = module(x)
            if isinstance(module, torch.nn.modules.linear.Linear):
                x.register_hook(self.compute_neuron_rank)
                self.n_activations.append(x)
                self.n_activation_to_layer[activation_index] = layer
                activation_index += 1
        return x
    def compute_filter_rank(self, grad):
        activation_index = len(self.activations) - self.grad_index - 1
        activation = self.activations[activation_index]

        taylor = activation * grad
        taylor = taylor.mean(dim=(0, 2, 3)).data

        if activation_index not in self.filter_ranks:
            self.filter_ranks[activation_index] = \
                torch.FloatTensor(activation.size(1)).zero_()

            if torch.cuda.is_available():
                self.filter_ranks[activation_index] = self.filter_ranks[activation_index].cuda()

        self.filter_ranks[activation_index] += taylor
        self.grad_index += 1

    def compute_neuron_rank(self, grad):
        activation_index = len(self.n_activations) - self.n_grad_index - 1
        activation = self.n_activations[activation_index]

        taylor = activation * grad
        taylor = taylor.mean(dim=(0)).data


        if activation_index not in self.neuron_ranks:
            self.neuron_ranks[activation_index] = \
                torch.FloatTensor(activation.size(1)).zero_()

            if torch.cuda.is_available():
                self.neuron_ranks[activation_index] = self.neuron_ranks[activation_index].cuda()

        self.neuron_ranks[activation_index] += taylor
        self.n_grad_index += 1

    def lowest_ranking_filters(self, num):
        data = []
        for i in sorted(self.filter_ranks.keys()):
            for j in range(self.filter_ranks[i].size(0)):
                data.append((self.activation_to_layer[i], j, self.filter_ranks[i][j]))
        return nsmallest(num, data, itemgetter(2))
    def lowest_ranking_neurons(self, num):
        data = []
        for i in sorted(self.neuron_ranks.keys()):
            if i != (len(self.n_activations) - 1):
                for j in range(self.neuron_ranks[i].size(0)):
                    data.append((self.n_activation_to_layer[i], j, self.neuron_ranks[i][j]))
        return nsmallest(num, data, itemgetter(2))
    def normalize_ranks_per_layer(self):
        filters = {}
        neurons = {}
        for i in self.filter_ranks:
            self.filter_ranks[i] = self.filter_ranks[i].cpu()
            v = torch.abs(self.filter_ranks[i])
            filters[i] = np.sqrt(torch.sum(v * v))
            v = v / np.sqrt(torch.sum(v * v))
            self.filter_ranks[i] = v.cpu()
        for i in self.neuron_ranks:
            self.neuron_ranks[i] = self.neuron_ranks[i].cpu()
            v = torch.abs(self.neuron_ranks[i])
            neurons[i] = np.sqrt(torch.sum(v * v))
            v = v / np.sqrt(torch.sum(v * v))
            self.neuron_ranks[i] = v.cpu()
        return filters, neurons
    def get_prunning_plan(self, num_filters_to_prune, num_neurons_to_prune):
        filters_to_prune = self.lowest_ranking_filters(num_filters_to_prune)
        neurons_to_prune = self.lowest_ranking_neurons(num_neurons_to_prune)

        # After each of the k filters are prunned,
        # the filter index of the next filters change since the model is smaller.
        filters_to_prune_per_layer = {}
        neurons_to_prune_per_layer = {}
        for (l, f, _) in filters_to_prune:
            if l not in filters_to_prune_per_layer:
                filters_to_prune_per_layer[l] = []
            filters_to_prune_per_layer[l].append(f)
            
        for (l, f, _) in neurons_to_prune:
            if l not in neurons_to_prune_per_layer:
                neurons_to_prune_per_layer[l] = []
            neurons_to_prune_per_layer[l].append(f)

        for l in filters_to_prune_per_layer:
            filters_to_prune_per_layer[l] = sorted(filters_to_prune_per_layer[l])
            for i in range(len(filters_to_prune_per_layer[l])):
                filters_to_prune_per_layer[l][i] = filters_to_prune_per_layer[l][i] - i
        
        for l in neurons_to_prune_per_layer:
            neurons_to_prune_per_layer[l] = sorted(neurons_to_prune_per_layer[l])
            for i in range(len(neurons_to_prune_per_layer[l])):
                neurons_to_prune_per_layer[l][i] = neurons_to_prune_per_layer[l][i] - i
                
        filters_to_prune = []
        for l in filters_to_prune_per_layer:
            for i in filters_to_prune_per_layer[l]:
                filters_to_prune.append((l, i))
                
        neurons_to_prune = []
        for l in neurons_to_prune_per_layer:
            for i in neurons_to_prune_per_layer[l]:
                neurons_to_prune.append((l, i))

        return filters_to_prune, neurons_to_prune             

In [0]:
class PrunningFineTuner:
    def __init__(self, train_path, test_path, model):
        self.train_data_loader = train_path
        self.test_data_loader = test_path
        self.model = model
        self.criterion = torch.nn.CrossEntropyLoss()
        self.prunner = Prunner(self.model)
        self.prunned_layers = []
        self.layers_norm = []
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-4)
        self.scheduler = ReduceLROnPlateau(self.optimizer,'min',factor=0.5 ,patience=2, verbose =True)
    def test(self):
        self.model.eval()
        correct = 0
        total = 0
        for i, (batch, label) in enumerate((self.test_data_loader)):
            if torch.cuda.is_available():
                batch = batch.cuda()
            output = self.model(batch)
            pred = output.data.max(1)[1]
            correct += pred.cpu().eq(label).sum()
            total += label.size(0)
        print('Accuracy : %f %%' % (100 * correct.data.cpu().numpy() / total))
        self.model.train()
        return 100 * correct.data.cpu().numpy() / total
    def train(self, epoches=1, rank_filters=False, pretrain=False):
        self.model.to('cuda')
        self.model.train()
        for i in tqdm_notebook(range(epoches)):
            print("Epoch: ", i)
            run_loss=0
            for i, (batch, label) in enumerate(tqdm_notebook(self.train_data_loader)):
                batch = batch.to('cuda')
                label = label.to('cuda')
                self.optimizer.zero_grad()
                if rank_filters:
                    output = self.prunner.forward(batch)
                    loss = self.criterion(output, label)
                    run_loss += loss.item()
                    loss.backward()
                else:
                    loss = self.criterion(self.model(batch), label)
                    run_loss += loss.item()
                    loss.backward()
                    self.optimizer.step()
            print('loss = ', run_loss/self.train_data_loader.dataset.__len__())
            if pretrain == True:
                self.scheduler.step(run_loss)
            self.test()        
    def speed(self):
        self.model.eval()
        k = 0
        for i in range(10):
            for i, (batch, label) in enumerate((self.test_data_loader)):
                if torch.cuda.is_available():
                    batch = batch.cuda()
                save = time.time()
                output = self.model(batch)
                k += time.time() - save
        self.model.train()
        return k/10
    def get_candidates_to_prune(self, num_filters_to_prune, num_neurons_to_prune):
        self.prunner.reset()
        self.train(rank_filters = True)
        self.layers_norm.append(self.prunner.normalize_ranks_per_layer())
        return self.prunner.get_prunning_plan(num_filters_to_prune, num_neurons_to_prune)
    def total_num_filters(self):
        filters = 0
        for name, module in self.model.features._modules.items():
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                filters = filters + module.out_channels
        return filters
    def total_num_neurons(self):
        neurons = 0
        for name, module in self.model.classifier._modules.items():
            if isinstance(module, torch.nn.modules.linear.Linear):
                neurons = neurons + module.out_features
        return neurons
    def total_num_weights(self):
        a = 0
        for i in self.model.parameters():
            a += i.numel()
        return a
    def prune(self, prune_type = 'conv', percent = 0.1):
        self.model.to('cuda')
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.0001, momentum=0.9, weight_decay=1e-4)
        self.prunning_scores = pd.DataFrame(columns=['score_cuted','score_fine', 'weights_prunned', 'forward_time'])
        self.prunned_layers = []
        if prune_type == 'conv':
            prune_type = 0
        elif prune_type == 'neuron':
            prune_type = 1
        else:
            raise TypeError
        self.train()
        self.prunning_scores.loc[100] = self.test()
        self.prunning_scores.loc[100, 'weights_prunned'] = None
        self.prunning_scores.loc[100, 'forward_time'] = self.speed()
        self.model.train()
        for param in self.model.features.parameters():
            param.requires_grad = True
        for param in self.model.classifier.parameters():
            param.requires_grad = True

        number_of_filters = self.total_num_filters()
        number_of_neurons = self.total_num_neurons()
        number_of_weights = self.total_num_weights()
        if percent < 1: 
            num_filters_to_prune_per_iteration = int(number_of_filters * percent)
            num_neurons_to_prune_per_iteration = int(number_of_neurons * percent) 
        else:
            num_filters_to_prune_per_iteration = percent
            num_neurons_to_prune_per_iteration = percent
        if prune_type == 0:
            iterations = int(float(number_of_filters) / num_filters_to_prune_per_iteration)
            print('going to prune ',num_filters_to_prune_per_iteration, 'per itaration')
        elif prune_type == 1:
            iterations = int(float(number_of_neurons) / num_neurons_to_prune_per_iteration)
            print('going to prune ', num_neurons_to_prune_per_iteration, 'per iteration')

        for _ in range(iterations):
            print("Ranking filters.. ")
            prune_targets = self.get_candidates_to_prune(num_filters_to_prune_per_iteration * (1-prune_type),
                                                         num_neurons_to_prune_per_iteration * prune_type)
            layers_prunned = {}
            for layer_index, filter_index in prune_targets[prune_type]:
                if layer_index not in layers_prunned:
                    layers_prunned[layer_index] = 0
                layers_prunned[layer_index] = layers_prunned[layer_index] + 1 
            print("Layers that will be prunned", layers_prunned)
            print("Prunning.. ")
            model = self.model.cpu()
            were_weights = self.total_num_weights()
            for layer_index, filter_index in prune_targets[prune_type]:
                model = prune_layer(model, layer_index, filter_index, conv_flag=(prune_type==0))
            self.model = model
            if torch.cuda.is_available():
                self.model = self.model.cuda()
            self.optimizer = optim.SGD(self.model.parameters(), lr=0.0001, momentum=0.9, weight_decay=1e-4)
            if prune_type == 0:
                message = (100*float(self.total_num_filters()) / number_of_filters)
                print("Filters left", str(message))
            elif prune_type == 1:
                message = (100*float(self.total_num_neurons()) / number_of_neurons)
                print("Neurons left", str(message))
            part = 100*float(self.total_num_weights()) / number_of_weights
            self.prunning_scores.loc[part, 'weights_prunned'] = were_weights - self.total_num_weights()
            self.prunning_scores.loc[part, 'score_cuted'] = self.test()
            print("Fine tuning to recover from prunning iteration.")
            self.train(epoches = 2)
            self.prunning_scores.loc[part, 'score_fine'] = self.test()
            self.prunning_scores.loc[part, 'forward_time'] = self.speed()
        self.prunning_scores.to_csv('prunning_scores.csv')

In [0]:
class VGG(torch.nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = models.vgg16(pretrained=True).features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 10))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [73]:
def get_mnist(batch_size):
    trsnform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)
    return train_loader, test_loader
def get_cifar(batch_size):
    trsnform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
    transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=True, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=False, download=True,
        transform=transform_test), batch_size=batch_size, shuffle=True)
    return train_loader, test_loader
trainloader, testloader = get_cifar(500)

Files already downloaded and verified
Files already downloaded and verified


In [64]:
net = VGG()
net = net.to('cuda')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
process = PrunningFineTuner(trainloader, testloader, net)

In [0]:
process.train(90, pretrain=True)

In [107]:
process.prune(percent=0.033)

Epoch:  0



loss =  2.6621536412858406e-06
Accuracy : 92.170000 %

Accuracy : 92.170000 %


going to prune  139 per itaration
Ranking filters.. 


Epoch:  0



loss =  1.728666239505401e-06
Accuracy : 92.170000 %

Layers that will be prunned {28: 29, 24: 30, 26: 51, 17: 7, 19: 10, 14: 3, 21: 5, 5: 2, 12: 1, 10: 1}
Prunning.. 
Filters left 96.7092803030303
Accuracy : 91.490000 %
Fine tuning to recover from prunning iteration.


Epoch:  0


...

Epoch:  0



loss =  0.004618371729850769
Accuracy : 10.000000 %

Layers that will be prunned {19: 11, 21: 7, 28: 22, 26: 22, 14: 13, 2: 12, 24: 6, 7: 8, 17: 6, 10: 11, 5: 6, 0: 11, 12: 4}
Prunning.. 
Filters left 1.2784090909090908
Accuracy : 10.000000 %
Fine tuning to recover from prunning iteration.


Epoch:  0



loss =  0.004607383561134338
Accuracy : 10.000000 %
Epoch:  1



loss =  0.00460705557346344
Accuracy : 10.000000 %

Accuracy : 10.000000 %


In [0]:
from google.colab import files

files.download('prunning_scores.csv')